In [2]:
import pandas as pd

from sqlite3 import connect

from datetime import datetime

In [3]:
conn = connect(':memory:')

### Grouped Static Data

In [4]:
# Objective: Predicting Attrition
df_hr = pd.read_csv("./sample-data/hr_data.csv")
df_hr = df_hr.loc[:,(~df_hr.columns.isin({"Employee_Name"})) & (~(df_hr.columns.str.contains("Unnamed")))]
df_hr["index"] = df_hr.index
df_hr.to_sql("hr_data", conn)

In [5]:
df_hr.groupby("DeptID",as_index=False).apply(lambda x: x[x.TermReason.str.contains("N/A")].count())



pd.read_sql("""SELECT * FROM (SELECT COUNT(TermReason) AS 'Headcount Per Department'
               FROM hr_data WHERE TermReason LIKE '%N/A%' 
               GROUP BY DeptID
               ORDER BY COUNT(TermReason) DESC)
               LEFT JOIN (SELECT COUNT(*) AS Leavers FROM hr_data WHERE TermReason NOT LIKE '%N/A%' 
               GROUP BY DeptID
               ORDER BY COUNT(TermReason) DESC) """,conn)

pd.read_sql("""SELECT (CAST(leavers as float)/CAST(all_employees AS float) ) as attrition_rate_by_dept FROM (SELECT (SUM(CASE WHEN TermReason NOT LIKE '%N/A%' THEN DeptID ELSE NULL END)) as leavers,
    (SUM(CASE WHEN TermReason IS NOT NULL 
    THEN DeptID ELSE NULL END)) AS all_employees
    FROM hr_data GROUP BY DeptID)""", conn)

,attrition_rate_by_dept
0,0.300000
1,NaN
2,0.200000
3,0.300000
4,0.399038
5,0.156250


### Text/Time Series Data

In [6]:
# Objective: Identifying self-organizing groups in the data (unsupervised), Generation Tasks
df_esg = pd.read_csv("./sample-data/ESG_daily_news.csv")
df_esg = df_esg.sort_values("Date",ascending=True)
df_esg.to_sql("esg_data",conn)

In [12]:
def search_for_keywords(keywords):
    '''
    search_for_keywords takes a keywords argument 
    :param keywords list:
    :return: pd.DataFrame 
    '''
    query = f"""SELECT * FROM esg_data WHERE 
             {" OR ".join([f"(headline LIKE '%{keyword}%')" for keyword in keywords])}"""
    return pd.read_sql(query, conn)

In [14]:
# search_for_keywords(["China", "Asia"])['headline'].tolist()




In [15]:
def filtering_by_date(start_date=None,end_date=None):
    query = "SELECT * FROM esg_data "
    add_ons = []
    if start_date is not None:
        add_ons.append(f"Date > '{start_date}'")
    if end_date is not None:
        add_ons.append(f"Date < '{end_date}'")
    query += " WHERE " + " AND ".join(add_ons)
    return pd.read_sql(query,conn).drop(columns=["index"])
filtering_by_date(str(datetime(2021,1,1)).split()[0],
                  str(datetime(2022,1,1)).split()[0])

,Date,headline,text
0,2021-10-21,China City Builders Buck Drop in Property Prices,A key part of China’s credit market is doing b...
1,2021-10-24,Negotiators Edge Closer to Global Carbon Marke...,Nations are edging toward a deal that might cr...
2,2021-10-25,Rich Nations Fail to Meet Climate Target Befor...,Rich countries have failed to meet their pledg...
3,2021-10-26,Jet Fuel Surges in Price as Travel Restriction...,Jet fuel is back in a big way. The oil product...
4,2021-10-28,Trillion-Dollar Boom Rings Bubble-Trouble Alarm,The doing-well-by-doing-good conviction drivin...
5,2021-10-28,Citi Pitches $1 Billion Social Bond Amid Race ...,Citigroup Inc. is returning to the social bond...
6,2021-10-31,China’s Low Oil Stockpiles May Spark Refiners ...,China may be forced to start buying crude at e...
7,2021-11-01,Credit Agricole Creeps Up on JPMorgan in Green...,Credit Agricole passed BNP Paribas and gained ...
8,2021-11-02,Teva Breaks Record With $5 Billion Bond Sale,Teva Pharmaceutical is selling a record $5 bil...
9,2021-11-03,"Nations Fall Short on Climate Disclosures, BNE...",Just a small handful of governments have imple...


In [16]:
# Storing data in SQL